In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
!mkdir -p /content/CTC360/{fastq,trimmed,qc,bam,variants,figures,ref}
project_dir = "/content/CTC360"
print("Project directory created.")


Project directory created.


In [ ]:
!apt-get update
!apt-get install -y sra-toolkit fastqc bwa samtools fastp


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,143 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InReleas

In [ ]:
!cd /content
!wget -c https://github.com/broadinstitute/gatk/releases/download/4.4.0.0/gatk-4.4.0.0.zip
!unzip -o gatk-4.4.0.0.zip


--2025-11-15 16:15:36--  https://github.com/broadinstitute/gatk/releases/download/4.4.0.0/gatk-4.4.0.0.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/27452807/0bd42ce9-3dc6-4814-971c-264ebaf66d3d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-15T17%3A15%3A40Z&rscd=attachment%3B+filename%3Dgatk-4.4.0.0.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-15T16%3A15%3A37Z&ske=2025-11-15T17%3A15%3A40Z&sks=b&skv=2018-11-09&sig=F9fY%2FLEaVBEUvjPzODMwCMesFcBvIMnkzOjV74dhJNs%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MzIyNjkzNywibmJmIjoxNzYzMjIzMzM3LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi

In [ ]:
!cd /content/CTC360/ref

# Download only chromosome 12 FASTA (hg38)
!wget -c https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr12.fa.gz
!gunzip chr12.fa.gz
!mv chr12.fa hg38.fa

# Index reference
!bwa index hg38.fa
!samtools faidx hg38.fa
!gatk-4.4.0.0/gatk CreateSequenceDictionary -R hg38.fa


--2025-11-15 16:17:58--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr12.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42940449 (41M) [application/x-gzip]
Saving to: ‘chr12.fa.gz’

chr12.fa.gz         100%[===================>]  40.95M  26.8MB/s    in 1.5s    

2025-11-15 16:18:00 (26.8 MB/s) - ‘chr12.fa.gz’ saved [42940449/42940449]

[bwa_index] Pack FASTA... 1.07 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=266550618, availableWord=30755168
[BWTIncConstructFromPacked] 10 iterations done. 50732074 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 93723498 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 131930362 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 165884698 characte

In [ ]:
!cd /content/CTC360/fastq

# Download SRA files
!prefetch SRR13973704
!prefetch SRR13973878

# Convert to FASTQ
!fasterq-dump SRR13973704 -O .
!fasterq-dump SRR13973878 -O .

# Compress
!gzip SRR13973704_1.fastq SRR13973704_2.fastq
!gzip SRR13973878_1.fastq SRR13973878_2.fastq



2025-11-15T16:20:52 prefetch.2.11.3: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2025-11-15T16:20:52 prefetch.2.11.3: 1) Downloading 'SRR13973704'...
2025-11-15T16:20:52 prefetch.2.11.3: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2025-11-15T16:20:52 prefetch.2.11.3:  Downloading via HTTPS...
2025-11-15T16:21:14 prefetch.2.11.3:  HTTPS download succeed
2025-11-15T16:21:22 prefetch.2.11.3:  'SRR13973704' is valid
2025-11-15T16:21:22 prefetch.2.11.3: 1) 'SRR13973704' was downloaded successfully
2025-11-15T16:21:27 prefetch.2.11.3: 'SRR13973704' has 25 unresolved dependencies
2025-11-15T16:21:27 prefetch.2.11.3: 2) Downloading 'ncbi-acc:CM000663.1?vdb-ctx=refseq'...
2025-11-15T16:21:27 prefetch.2.11.3:  Downloading via HTTPS...
2025-11-15T16:21:28 prefetch.2.11.3:  HTTPS download succeed
2025-11-15T16:21:28 prefetch.2.11.3: 2) 'ncbi-acc:CM000663.1

In [ ]:
!mv /content/SRR13973704_1.fastq.gz /content/CTC360/fastq/CTC360_pre_R1.fastq.gz
!mv /content/SRR13973704_2.fastq.gz /content/CTC360/fastq/CTC360_pre_R2.fastq.gz

!mv /content/SRR13973878_1.fastq.gz /content/CTC360/fastq/CTC360_post_R1.fastq.gz
!mv /content/SRR13973878_2.fastq.gz /content/CTC360/fastq/CTC360_post_R2.fastq.gz


In [ ]:
%cd /content/CTC360/qc
!fastqc ../fastq/CTC360_pre_R1.fastq.gz ../fastq/CTC360_pre_R2.fastq.gz \
        ../fastq/CTC360_post_R1.fastq.gz ../fastq/CTC360_post_R2.fastq.gz


/content/CTC360/qc
Started analysis of CTC360_pre_R1.fastq.gz
Approx 5% complete for CTC360_pre_R1.fastq.gz
Approx 10% complete for CTC360_pre_R1.fastq.gz
Approx 15% complete for CTC360_pre_R1.fastq.gz
Approx 20% complete for CTC360_pre_R1.fastq.gz
Approx 25% complete for CTC360_pre_R1.fastq.gz
Approx 30% complete for CTC360_pre_R1.fastq.gz
Approx 35% complete for CTC360_pre_R1.fastq.gz
Approx 40% complete for CTC360_pre_R1.fastq.gz
Approx 45% complete for CTC360_pre_R1.fastq.gz
Approx 50% complete for CTC360_pre_R1.fastq.gz
Approx 55% complete for CTC360_pre_R1.fastq.gz
Approx 60% complete for CTC360_pre_R1.fastq.gz
Approx 65% complete for CTC360_pre_R1.fastq.gz
Approx 70% complete for CTC360_pre_R1.fastq.gz
Approx 75% complete for CTC360_pre_R1.fastq.gz
Approx 80% complete for CTC360_pre_R1.fastq.gz
Approx 85% complete for CTC360_pre_R1.fastq.gz
Approx 90% complete for CTC360_pre_R1.fastq.gz
Approx 95% complete for CTC360_pre_R1.fastq.gz
Analysis complete for CTC360_pre_R1.fastq.gz
S

In [ ]:
%cd /content/CTC360

!fastp \
  -i fastq/CTC360_pre_R1.fastq.gz \
  -I fastq/CTC360_pre_R2.fastq.gz \
  -o trimmed/CTC360_pre_R1.trimmed.fastq.gz \
  -O trimmed/CTC360_pre_R2.trimmed.fastq.gz \
  --html qc/pre_fastp.html \
  --json qc/pre_fastp.json

!fastp \
  -i fastq/CTC360_post_R1.fastq.gz \
  -I fastq/CTC360_post_R2.fastq.gz \
  -o trimmed/CTC360_post_R1.trimmed.fastq.gz \
  -O trimmed/CTC360_post_R2.trimmed.fastq.gz \
  --html qc/post_fastp.html \
  --json qc/post_fastp.json


/content/CTC360
Read1 before filtering:
total reads: 8059669
total bases: 1192376576
Q20 bases: 1130846475(94.8397%)
Q30 bases: 1080570668(90.6233%)

Read2 before filtering:
total reads: 8059669
total bases: 1191862806
Q20 bases: 1112656685(93.3544%)
Q30 bases: 1037810018(87.0746%)

Read1 after filtering:
total reads: 7863888
total bases: 1162674436
Q20 bases: 1109425334(95.4201%)
Q30 bases: 1061472467(91.2958%)

Read2 aftering filtering:
total reads: 7863888
total bases: 1162224972
Q20 bases: 1096493049(94.3443%)
Q30 bases: 1024828287(88.1781%)

Filtering result:
reads passed filter: 15727776
reads failed due to low quality: 391526
reads failed due to too many N: 36
reads failed due to too short: 0
reads with adapter trimmed: 80234
bases trimmed due to adapters: 1769457

Duplication rate: 19.3541%

Insert size peak (evaluated by paired-end reads): 167

JSON report: qc/pre_fastp.json
HTML report: qc/pre_fastp.html

fastp -i fastq/CTC360_pre_R1.fastq.gz -I fastq/CTC360_pre_R2.fastq.gz -

In [ ]:
!bwa mem -t 8 /content/hg38.fa \
  /content/CTC360/trimmed/CTC360_pre_R1.trimmed.fastq.gz \
  /content/CTC360/trimmed/CTC360_pre_R2.trimmed.fastq.gz \
  | samtools sort -o /content/CTC360/bam/CTC360_pre.bam

!bwa mem -t 8 /content/hg38.fa \
  /content/CTC360/trimmed/CTC360_post_R1.trimmed.fastq.gz \
  /content/CTC360/trimmed/CTC360_post_R2.trimmed.fastq.gz \
  | samtools sort -o /content/CTC360/bam/CTC360_post.bam


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 541552 sequences (80000010 bp)...
[M::process] read 541052 sequences (80000234 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 1525, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (29, 68, 102)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 248)
[M::mem_pestat] mean and std.dev: (72.48, 47.52)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 321)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 541552 reads in 1053.462 CPU sec, 628.793 real sec
[M::process] read 540758 sequences (80000200 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (3, 8158, 1, 1)
[M::mem_pestat] skip orientation 

In [ ]:
!ls -lh bam

total 1.6G
-rw-r--r-- 1 root root 249M Nov 15 21:02 CTC360_post.bam
-rw-r--r-- 1 root root 1.4G Nov 15 20:29 CTC360_pre.bam


In [ ]:
!samtools index /content/CTC360/bam/CTC360_pre.bam
!samtools index /content/CTC360/bam/CTC360_post.bam

In [ ]:
%cd /content/CTC360

!/content/gatk-4.4.0.0/gatk MarkDuplicates \
  -I bam/CTC360_pre.bam \
  -O bam/CTC360_pre.dedup.bam \
  -M bam/CTC360_pre.metrics \
  --REMOVE_DUPLICATES true

!/content/gatk-4.4.0.0/gatk MarkDuplicates \
  -I bam/CTC360_post.bam \
  -O bam/CTC360_post.dedup.bam \
  -M bam/CTC360_post.metrics \
  --REMOVE_DUPLICATES true

!samtools index bam/CTC360_pre.dedup.bam
!samtools index bam/CTC360_post.dedup.bam

!ls -lh bam


/content/CTC360
Using GATK jar /content/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /content/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar MarkDuplicates -I bam/CTC360_pre.bam -O bam/CTC360_pre.dedup.bam -M bam/CTC360_pre.metrics --REMOVE_DUPLICATES true
21:04:00.215 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/content/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
[Sat Nov 15 21:04:00 UTC 2025] MarkDuplicates --INPUT bam/CTC360_pre.bam --OUTPUT bam/CTC360_pre.dedup.bam --METRICS_FILE bam/CTC360_pre.metrics --REMOVE_DUPLICATES true --MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP 50000 --MAX_FILE_HANDLES_FOR_READ_ENDS_MAP 8000 --SORTING_COLLECTION_SIZE_RATIO 0.25 --TAG_DUPLICATE_SET_MEMBERS false --REMOVE_SEQUENCING_DUPLICATES false --TAGGING_POLICY DontT

In [ ]:
!ls -lh /content/hg38.fa*


-rw-r--r-- 1 root root  462 Nov 15 16:19 /content/hg38.fa.amb
-rw-r--r-- 1 root root   45 Nov 15 16:19 /content/hg38.fa.ann
-rw-r--r-- 1 root root 128M Nov 15 16:19 /content/hg38.fa.bwt
-rw-r--r-- 1 root root  32M Nov 15 16:19 /content/hg38.fa.pac
-rw-r--r-- 1 root root  64M Nov 15 16:20 /content/hg38.fa.sa


In [ ]:

!ls -lh /content/CTC360


total 130M
drwxr-xr-x 2 root root 4.0K Nov 15 21:07 bam
drwxr-xr-x 2 root root 4.0K Nov 15 16:56 fastq
drwxr-xr-x 2 root root 4.0K Nov 15 16:13 figures
-rw-r--r-- 1 root root   98 Nov 15 16:20 hg38.dict
-rw-r--r-- 1 root root 130M Jan 24  2014 hg38.fa
-rw-r--r-- 1 root root   24 Nov 15 16:20 hg38.fa.fai
drwxr-xr-x 2 root root 4.0K Nov 15 16:47 qc
drwxr-xr-x 2 root root 4.0K Nov 15 16:13 ref
drwxr-xr-x 2 root root 4.0K Nov 15 17:01 trimmed
drwxr-xr-x 2 root root 4.0K Nov 15 16:13 variants


In [ ]:
!ls -lh /content/CTC360/variants


total 0


In [ ]:
!ls -lh /content/CTC360/hg38.fa


-rw-r--r-- 1 root root 130M Jan 24  2014 /content/CTC360/hg38.fa


In [ ]:
!ls -lh /content/CTC360/variants/


total 0


In [ ]:
!samtools view -H /content/CTC360/bam/CTC360_pre.dedup.bam


@HD	VN:1.6	SO:coordinate
@SQ	SN:chr12	LN:133275309
@PG	ID:bwa	PN:bwa	VN:0.7.17-r1188	CL:bwa mem -t 8 /content/hg38.fa /content/CTC360/trimmed/CTC360_pre_R1.trimmed.fastq.gz /content/CTC360/trimmed/CTC360_pre_R2.trimmed.fastq.gz
@PG	ID:samtools	PN:samtools	PP:bwa	VN:1.13	CL:samtools sort -o /content/CTC360/bam/CTC360_pre.bam
@PG	ID:MarkDuplicates	VN:Version:4.4.0.0	CL:MarkDuplicates --INPUT bam/CTC360_pre.bam --OUTPUT bam/CTC360_pre.dedup.bam --METRICS_FILE bam/CTC360_pre.metrics --REMOVE_DUPLICATES true --MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP 50000 --MAX_FILE_HANDLES_FOR_READ_ENDS_MAP 8000 --SORTING_COLLECTION_SIZE_RATIO 0.25 --TAG_DUPLICATE_SET_MEMBERS false --REMOVE_SEQUENCING_DUPLICATES false --TAGGING_POLICY DontTag --CLEAR_DT true --DUPLEX_UMI false --FLOW_MODE false --FLOW_QUALITY_SUM_STRATEGY false --USE_END_IN_UNPAIRED_READS false --USE_UNPAIRED_CLIPPED_END false --UNPAIRED_END_UNCERTAINTY 0 --FLOW_SKIP_FIRST_N_FLOWS 0 --FLOW_Q_IS_KNOWN_END false --FLOW_EFFECTIVE_QUALITY_THRESHO

In [ ]:
!/content/gatk-4.4.0.0/gatk AddOrReplaceReadGroups \
    -I /content/CTC360/bam/CTC360_pre.dedup.bam \
    -O /content/CTC360/bam/CTC360_pre.RG.bam \
    -RGID PRE \
    -RGLB lib1 \
    -RGPL ILLUMINA \
    -RGPU unit1 \
    -RGSM PRE


Using GATK jar /content/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /content/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar AddOrReplaceReadGroups -I /content/CTC360/bam/CTC360_pre.dedup.bam -O /content/CTC360/bam/CTC360_pre.RG.bam -RGID PRE -RGLB lib1 -RGPL ILLUMINA -RGPU unit1 -RGSM PRE
21:29:53.897 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/content/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
[Sat Nov 15 21:29:53 UTC 2025] AddOrReplaceReadGroups --INPUT /content/CTC360/bam/CTC360_pre.dedup.bam --OUTPUT /content/CTC360/bam/CTC360_pre.RG.bam --RGID PRE --RGLB lib1 --RGPL ILLUMINA --RGPU unit1 --RGSM PRE --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 2 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREAT

In [ ]:
!samtools index /content/CTC360/bam/CTC360_pre.RG.bam



In [ ]:
!samtools view -H /content/CTC360/bam/CTC360_pre.RG.bam | grep "@RG"


@RG	ID:PRE	LB:lib1	PL:ILLUMINA	SM:PRE	PU:unit1


In [ ]:
!/content/gatk-4.4.0.0/gatk Mutect2 \
  -R /content/CTC360/hg38.fa \
  -I /content/CTC360/bam/CTC360_pre.RG.bam \
  -tumor PRE \
  -O /content/CTC360/variants/CTC360_pre.unfiltered.vcf.gz


Using GATK jar /content/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /content/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar Mutect2 -R /content/CTC360/hg38.fa -I /content/CTC360/bam/CTC360_pre.RG.bam -tumor PRE -O /content/CTC360/variants/CTC360_pre.unfiltered.vcf.gz
22:06:33.006 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/content/gatk-4.4.0.0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
22:06:33.092 INFO  Mutect2 - ------------------------------------------------------------
22:06:33.099 INFO  Mutect2 - The Genome Analysis Toolkit (GATK) v4.4.0.0
22:06:33.099 INFO  Mutect2 - For support and documentation go to https://software.broadinstitute.org/gatk/
22:06:33.099 INFO  Mutect2 - Executing as root@4451a6451ba3 on Linux v6.6.105+ amd64
22:06:33.099 INFO  M